In [ ]:
# install pycomm3 and ipywidgets packages
!pip install -q pycomm3 ipywidgets

In [6]:
import pycomm3

# Discover the devices on the network
devices_list = pycomm3.CIPDriver.discover()
print(devices_list)

#! if no devices are found, it will return an empty list: []
#! it should return something list below when the PLC is connected to the laptop via ethernet

# [{'encap_protocol_version': 1, 'ip_address': '10.10.0.120', 'vendor': 'Rockwell Automation/Allen-Bradley',
#   'product_type': 'Communications Adapter', 'product_code': 185, 'revision': {'major': 2, 'minor': 7},
#   'status': b'T\x00', 'serial': 'aabbcdd', 'product_name': '1763-L16BWA B/7.00', 'state': 0},
#  {'encap_protocol_version': 1, 'ip_address': '10.10.1.100', 'vendor': 'Rockwell Automation/Allen-Bradley',
#   'product_type': 'Communications Adapter', 'product_code': 191, 'revision': {'major': 20, 'minor': 19},
#   'status': b'0\x00', 'serial': 'eeffgghh', 'product_name': '1769-L23E-QBFC1 Ethernet Port', 'state': 3}]

[]


In [ ]:
class PumpController:
    """
    A PumpController class to handle communication with a pump using EtherNet/IP 
    protocol. It allows reading and writing parameters to the pump.
    """
    def __init__(self, ip_address):
        """
        Initializes the PumpController with the given IP address of the pump.

        Parameters:
        ip_address (str): The IP address of the pump.
        """
        self.ip_address = ip_address

        #! potential error here, don't how the tag list is initialized automatically or not
        #! or maybe the pump don't work with this package at all, then we have to fall back to the CIPDriver
        with pycomm3.LogixDriver(ip_address, init_tags=True) as pump:
            self.pump = pump
        #! potential error here, don't how the tag list is initialized automatically or not

        # I don't if open() is necessary here, official doc doesn't mention it
        open_return = self.pump.open()
        print(f"Open return: {open_return}")

        # Check if the connection was successful
        if self.pump.connected:
            print(f"Connected to pump at {ip_address}")

            # Print PLC info
            plc_info = self.pump.get_plc_info()

            print("PLC Info:")
            for key, value in plc_info.items():
                print(f"{key}: {value}")
        else:
            print(f"Failed to connect to pump at {ip_address}")

# if the device is not empty, we will attempt to create a PumpController object for each device
for device in devices_list:
    ip_address = device['ip_address']
    pump_controller = PumpController(ip_address)